# <center>Taxonomic Classification</center>

## Taxonomic Classification Methods for Metagenomic Data

1. Mothur
2. Dada2
3. Qiime
4. OneCodex Target Loci Analysis



OTU PICKING STRATEGIES
https://github.com/biocore/qiime/blob/master/doc/tutorials/otu_picking.rst

QIIME PIPELINE
1. OTU picking: open-reference OTU picking
2. Identify chimeric sequences with Uchime (default)
3. Taxonomy assignment with RDP (because good for medical)
4. ALIGN SEQUENCES FOR PHYLOGENIC TREE using Pynast
5. Infer phylogenetic tree using FastTree

## Pipeline

#### Server 
1. All analyses run on scg4.
2. qlogin -l h_vmem=10G
3. Most jobs submitted with qsub, not in interactive 

#### Data


### Preprocessing

#### Step 1: Download data using fastq-dump

Downloaded 35 fastq files of 16S from SRA - each matches a one of 24 patients for which we have chart data

```bash
for foo in $(cat ~/bhatt/margaret/mgh/sure.txt); do

fastq-dump $foo
echo $foo

done
```

SRR1196422
SRR1196501
SRR1196676
SRR1196682
SRR1196684
SRR1196714
SRR1196732
SRR1196775
SRR1196776
SRR1196786
SRR1196826
SRR1196876
SRR1196880
SRR1196897
SRR1196901
SRR1196902
SRR1196924
SRR1196930
SRR1201469
SRR1201508
SRR1201509
SRR1201518
SRR1201576
SRR1201585
SRR1196333
SRR1196334
SRR1196336
SRR1196343
SRR1196351
SRR1196399
SRR1196416
SRR1196966
SRR1196974
SRR1201472
SRR1201557

#### Step 2: Run Fastqc on all samples

Just ran it in interactive qlogin instead of qsub since so few samples

```bash
PROJECT_PATH="/srv/gsfs0/projects/bhatt/margaret/mgh"
DATA_PATH="/srv/gsfs0/projects/bhatt/margaret/mgh/raw"
module load fastqc/0.11.2
fastqc --extract -t 10 --outdir=$PROJECT_PATH/fastqc/ $DATA_PATH/*fastq
```

#### Step 3: Check quality of samples
Go into directory with fastqc results and check which ones failed. The summary.txt file in each directory created by FastQC for each sample has cat-able results
```bash
cat SRR*_fastqc/summary.txt | grep FAIL > failed_fastqc.txt

# Number that failed "Per base sequence quality" test
cat failed_fastqc.txt | grep quality | wc -l
# in this run: 35/35

# Number that failed "Per sequence GC content" test
cat failed_fastqc.txt | grep GC\ \content | wc -l
# in this run: 30/35

# Number that failed "Per base sequence content" test
cat failed_fastqc.txt | grep sequence\ \content | wc -l
# in this run: 35/35

# Number that failed "Kmer Content" test
cat failed_fastqc.txt | grep Kmer\ \Content | wc -l
# in this run: 30/35

# Number that failed "Sequence Duplication Levels" test
cat failed_fastqc.txt | grep Sequence\ \Duplication\ \Levels | wc -l
# in this run: 35/35

# Number that failed "Overrepresented sequences" test
cat failed_fastqc.txt | grep Overrepresented\ \sequences | wc -l
# in this run: 35/35

```

#### Step 4: Deduplication using super_deduper
```bash
cat ~/bhatt/margaret/mgh/raw/samples_mgh.txt | xargs -n 3 -I foo sh -c "~/bhatt/moss/tools/qc/Super-Deduper/super_deduper -s 5 -l 90 -p ~/bhatt/margaret/mgh/dedup/foo -U foo.fastq"
```

#### Step 5: Trim adapter sequences with trim_galore (trim and fastqc wrapper)
cutadapt 1.7.1 with Python 2.7.6

Determining trimming parameters
```bash
cat SRR*_fastqc/fastqc_data.txt | grep Per\ \base\ \sequence\ \quality -A 11
```

```bash
trim_galore -q 20 --fastqc --length 250 -o $PROJECT_PATH/trim/trimmed_"$foo" -U $DATA_PATH/"$foo"_nodup_PE1.fastq
```

Get read length distribution for sample foo
```bash
cat "$foo".fastq | perl -ne '$s=<>;<>;<>;chomp($s);print length($s)."\n";' >  "$foo".readslength.txt
```

How many sequences were removed because of the 250 bp (75% of original length of 350bp) length cutoff:
```bash
cat trimmed_SRR1*/*report.txt | grep Sequences\ \removed
```

#### Step 6: Reassess Sequence Quality after Trimming

1. Adapter Sequence Removal: Used default Universal Illumina Adapter. PASS on all after removal
2. Concatenated all FAIL tests into one file for easy grepping
3. Common FAIL tests in trimmed+deduped reads
4. Deduplication with Super-Deduper seems to have had little effect. 28/35 samples still have high duplication levels.
5. Looking at top 5 overrepresented sequences: representation 2-13% of the sample
6. For FAILED Kmer tests, anywhere from 5 to 830 counts of overrepresented KMER
7. Up to 3182 count for sequences with GC content FAIL

```bash
# Number that failed "Per base sequence quality" test
cat failed_fastqc.txt | grep quality | wc -l
# in this run: 14/35

# Number that failed "Per sequence GC content" test
cat failed_fastqc.txt | grep GC\ \content | wc -l
# in this run: 12/35

# Number that failed "Per base sequence content" test
cat failed_fastqc.txt | grep sequence\ \content | wc -l
# in this run: 35/35

# Number that failed "Kmer Content" test
cat failed_fastqc.txt | grep Kmer\ \Content | wc -l
# in this run: 28/35

# Number that failed "Sequence Duplication Levels" test
cat failed_fastqc.txt | grep Sequence\ \Duplication\ \Levels | wc -l
# in this run: 28/35

# Number that failed "Overrepresented sequences" test
cat failed_fastqc.txt | grep Overrepresented\ \sequences | wc -l
# in this run: 35/35

cat *fastqc/*summary.txt |grep FAIL >fails.txt
cat *fastqc/fastqc_data.txt | grep Overrepresented -A 2 |sort -k 3 -n
cat *fastqc/fastqc_data.txt | grep Kmer -A 4 | sort -k 2 -n

```

## OTU binning and classification with QIIME

[Tutorial for reference](http://nbviewer.jupyter.org/github/biocore/qiime/blob/1.9.1/examples/ipynb/illumina_overview_tutorial.ipynb)
#### Step 1: Set up Qiime environment
```bash
# Install Anaconda http://conda.pydata.org/docs/install/quick.html#os-x-miniconda-install
# Download installer and run bash script

# Install Qiime http://qiime.org/install/install.html
conda create -n qiime1 python=2.7 qiime matplotlib=1.4.3 mock nose -c bioconda

# Check installation
conda list
conda update

# To activate this environment, use:
source activate qiime1

# To deactivate this environment, use:
source deactivate qiime1
```
#### Step 2: Convert fastq files to fasta 

```bash
# Download libgtextutils (dependency of fastx_toolkit)
curl -O https://github.com/agordon/libgtextutils/releases/download/0.7/libgtextutils-0.7.tar.gz
gunzip libgtextutils-0.7.tar.gz
tar xvf libgtextutils-0.7.tar
cd libgtextutils-0.7
./configure
make
sudo make install

# Download fastx-toolkit
curl -O https://github.com/agordon/fastx_toolkit/releases/download/0.0.14/fastx_toolkit-0.0.14.tar.bz2
tar xvf fastx_toolkit-0.0.14.tar.bz2
cd fastx_toolkit-0.0.14
# Make sure pkg-utils is up to date, if not then 
# brew install pkg-config
export PKG_CONFIG_PATH=/usr/local/lib/pkgconfig:$PKG_CONFIG_PATH
./configure
make
sudo make install
# fastx toolkit should now be installed

# Run fastx tool fastq_to_fasta - done in bash
fastq_to_fasta -i SRRXXXXX.fq -o SRRXXXXX.fa

# Look at fasta files (after trim_galore and deduping, but before concatenation for Qiime)
# In directory with fasta files, get number of seqs in each file:
# grep -c "^>" * 

SRR1196422	22366
SRR1196501	22989
SRR1196676	10205
SRR1196682	41048
SRR1196684	23049
SRR1196714	35192
SRR1196732	3078
SRR1196775	32083
SRR1196776	5356
SRR1196786	8378
SRR1196826	21424
SRR1196876	43310
SRR1196880	21040
SRR1196897	51981
SRR1196901	17216
SRR1196902	43941
SRR1196924	17290
SRR1196930	76677
SRR1201469	6345
SRR1201508	32424
SRR1201509	1714
SRR1201518	6323
SRR1201576	67603
SRR1201585	43558
```

#### Step 3: Label sequences by sample from which they originated and merge all samples into one file

If barcodes are used, then this is naturally uneccessary, however, starting with samples that are individual fastq/fasta files, this is a solution for accetable input into qiime tools, such as core_diversity_analysis.py

[See Google Qiime Forum for reference](https://groups.google.com/forum/#!topic/qiime-forum/XDQnB_QPfHI)

```bash
# Create map text file mapping.txt

    
# add_qiime_labels.py -m mapping.txt -i fasta/ -c InputFileColName -o fastaForQiime/

add_qiime_labels.py -m mapping.txt -i fasta/ -c InputFileName -o fastaForQiime/

# Now have combined_seqs.fna in fastaForQiime/

# Count number of sequences in combined file
count_seqs.py -i fastaForQiime/combined_seqs.fna
#654590  : fastaForQiime/combined_seqs.fna (Sequence lengths (mean +/- std): 323.4642 +/- 43.4627)
#654590  : Total
```

#### Step 4: Pick open reference OTUs
```bash
# Pick open reference OTUs (takes many minutes...maybe an hour)
pick_open_reference_otus.py -o otus/ -i fastaForQiime/combined_seqs.fna -p ../uc_fast_params.txt

# Summaryze BIOM table

biom summarize-table -i otus/otu_table_mc2_w_tax_no_pynast_failures.biom
#Num samples: 24
#Num observations: 219
#Total count: 2310
#Table density (fraction of non-zero values): 0.086

#Counts/sample summary:
 #Min: 1.0
 #Max: 721.0
 #Median: 58.000
 #Mean: 96.250
 #Std. dev.: 146.039
 #Sample Metadata Categories: None provided
 #Observation Metadata Categories: taxonomy

#Counts/sample detail:
#SRR1196924: 1.0
#SRR1201509: 2.0
#SRR1201469: 3.0
#SRR1196732: 13.0
#SRR1196786: 13.0
#SRR1196501: 14.0
#SRR1196897: 22.0
#SRR1196422: 25.0
#SRR1201518: 27.0
#SRR1201508: 32.0
#SRR1196684: 41.0
#SRR1196776: 58.0
#SRR1196880: 58.0
#SRR1201576: 60.0
#SRR1196901: 66.0
#SRR1196775: 69.0
#SRR1196826: 91.0
#SRR1196876: 100.0
#SRR1196676: 110.0
#SRR1201585: 152.0
#SRR1196902: 166.0
#SRR1196930: 223.0
#SRR1196714: 243.0
#SRR1196682: 721.0

```
#### Step 5: Diversity analysis
Probably shouldn't do this because of low counts in biom table
```bash
core_diversity_analyses.py -o cdout/ -i otus/otu_table_mc2_w_tax_no_pynast_failures.biom -m files/mapping.txt -t otus/rep_set.tre -e 50
# All classifications UNKNOWN. Because of low count in Step 4. But why?

```

***

# Dada2

[See tutorial here](http://benjjneb.github.io/dada2/tutorial.html)

```bash
# Install
source("https://bioconductor.org/biocLite.R")
biocLite("dada2")
library(dada2); packageVersion("dada2")
library(ShortRead); packageVersion("ShortRead")
library(ggplot2); packageVersion("ggplot2")

> path<-"../raw/"
> fns<-list.files(path)
> fns
 [1] "SRR1196422.fastq" "SRR1196501.fastq"
 [3] "SRR1196676.fastq" "SRR1196682.fastq"
 [5] "SRR1196684.fastq" "SRR1196714.fastq"
 [7] "SRR1196732.fastq" "SRR1196775.fastq"
 [9] "SRR1196776.fastq" "SRR1196786.fastq"
[11] "SRR1196826.fastq" "SRR1196876.fastq"
[13] "SRR1196880.fastq" "SRR1196897.fastq"
[15] "SRR1196901.fastq" "SRR1196902.fastq"
[17] "SRR1196924.fastq" "SRR1196930.fastq"
[19] "SRR1201469.fastq" "SRR1201508.fastq"
[21] "SRR1201509.fastq" "SRR1201518.fastq"
[23] "SRR1201576.fastq" "SRR1201585.fastq"

```